# Pytorch — фреймворк для обучения нейронных сетей и не только

## Зачем нужны библиотеки для обучения нейронных сетей?

* иметь качественные и эффективные реализации лоссов, слоев, оптимизаторов
* стандартизация кода, использование сторонних библиотек/моделей/имплементаций статей от коммьюнити
* ускорение с помощью GPU без написания специального кода

## Как обучаюся нейронные сети?

- методы оптимизации, использующие градиент

- для оптимизации вычисления градиента используют метод обратного распространения ошибки

Поэтому в основе любого фреймворка лежит автоматическое дифференцирование

In [0]:
%pylab inline
import torch

## torch.nn, torch.optim

Удобство библиотеки не ограничивается одним только автодифференцированием. Многое из того что мы написали выше уже реализовано.

В частности, стандартные слои, функции активаций, лоссы, оптимизаторы. Вот как выгдядит упрощенная версия кода, использующая модуль torch.nn и torch.optim

## nn.Module, nn.Dataset

<img src="https://github.com/data-mining-in-action/DMIA_Industry_2019_Autumn/blob/master/seminar04/pytorch_tutorial/images/fashion-mnist-sprite.png?raw=true" width="600">
</br>

Теперь попробуем обучить классификатор одежды для датасета FashionMNIST (https://github.com/zalandoresearch/fashion-mnist ). 

Основные новые концепции, которые мы усвоим на этом примере — **Dataset,  Dataloader, Model**. Они необходимы для удобной работы с библиотекой.

In [0]:
import numpy as np
from urllib import request
import gzip
import pickle
import matplotlib.pyplot as plt
import itertools
    
# вспомогательные функции для загрузки данных

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]
# mnist loading code from https://github.com/hsjeong5/MNIST-for-Numpy
def download_mnist():
    base_url = "http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]


In [0]:
from torch.utils.data import DataLoader, Dataset
from pathlib import Path

import torchvision
import torchvision.transforms as transforms

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [0]:
# Начнем с класса Dataset. По названию можно догадаться, что он нужен для работы с датасетами
# Это очень удобная абстракция, которая в дальнейшем может быть использована в комбинации с 
# Dataloader, для параллельной загрузки данных. Но об этом позже.

# Кастомный датасет должен переопределить два метода: __len__ и __getitem__. 

class FashionMNISTDataset(Dataset):
    def __init__(self, transforms=None, training=True):
        if not Path("mnist.pkl").exists():
            download_mnist()
            save_mnist()
        train_imgs, train_labels, test_imgs, test_labels = load()
        self.training = training
        self.transforms = transforms
        
        if self.training:
            self.imgs = train_imgs
            self.labels = train_labels
        else:
            self.imgs = test_imgs
            self.labels = test_labels
            
    def __getitem__(self, idx):
        img = self.imgs[idx].reshape((-1, 28, 1))
        if self.transforms is not None:
            img = self.transforms(img)
        return img, torch.tensor(self.labels[idx], dtype=torch.long)
    
    def __len__(self):
        return len(self.imgs)

In [0]:
classes = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
    'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
data_transformation = transforms.Compose([transforms.ToTensor()])
datasets = {
    x: FashionMNISTDataset(
        training=(x == "train"), 
        transforms=data_transformation) for x in ["train", "test"]}

In [0]:
image, label = datasets["train"][1]
print(image.shape, label)
plt.title(classes[label.numpy()])
plt.imshow(image.numpy().squeeze(), cmap="gray")

In [0]:
# DataLoader это python iterable, который возвращает элементы Dataset бачами

dataloaders = {
    x: DataLoader(datasets[x], batch_size=10, shuffle=True, num_workers=0)
    for x in ["train", "test"]}
batch = next(iter(dataloaders["train"]))
images, labels = batch
print(images.shape)

In [0]:
grid = torchvision.utils.make_grid(images, nrow=10)

plt.figure(figsize=(15,15))
plt.imshow(np.transpose(grid, (1,2,0)))

print('labels:', labels)

# Настало время обучить сверточную нейронную сеть!

Пример архитектуры сверточной сети

![alt text](https://miro.medium.com/max/3288/1*uAeANQIOQPqWZnnuH-VEyw.jpeg)

## Сейчас мы реализуем двухслойную сверточную нейронную сеть и обучим ее для задачи классифиции картинок

Какие слои будем использовать сегодня:
- Сверточный (`nn.Conv2d`)
- функцию активации ReLU (`nn.ReLU`)
- max-пуллинг (`nn.MaxPool2d`)
- полносвязный (линейный) слой (`nn.Linear`)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

# Модели в PyTorch наследуются от nn.Module
# В стандартном случае нужно определить только __init__ для описания
# состовляющих частей архитектуры и forward для определения того, 
# как они должны взаимодействовать при прямом проходе

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.layer1 = nn.Sequential(
            # свертка
            # нелинейность
            # пуллинг
            # ВАШ КОД ЗДЕСЬ
            )
        
        self.layer2 = nn.Sequential(
            # ВАШ КОД ЗДЕСЬ
            )
        
        self.linear = # линейный слой с выходом на 10 классов
        # ВАШ КОД ЗДЕСЬ


    def forward(self, x):
        # реализуйте проход картинки по сети вперед
        # используя уже созданные выше слои
        return out

In [0]:
device = "cpu"
model = Network().to(device)
print(model)
print("\nModel parameters shapes:")
for name, parameter in model.named_parameters():
    print(name, parameter.shape)

In [0]:
def train(model, dataloader, n_epoch):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(1, n_epoch + 1):
        for batch_id, (image, label) in enumerate(dataloader):
            label, image = label.to(device), image.to(device)
            output = model(image)
            loss = criterion(output, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if batch_id % 1000 == 0:
                print('Loss :{:.4f} Epoch[{}/{}]'.format(loss.item(), epoch, n_epoch))
    return model

In [0]:
LR = 0.01
n_epoch = 5
dataloaders = {
    x: DataLoader(datasets[x], batch_size=1000, shuffle=x == "train")
    for x in ["train", "test"]}

model = train(model, dataloader=dataloaders["train"], n_epoch=n_epoch)

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score

def test(model, dataloader):
    predictions = []
    targets = []
    model.eval()
    with torch.no_grad():
        for image, label in dataloader:
            image = image.to(device)
            label = label.to(device)
            outputs = model(image)
            predicted = torch.argmax(outputs,dim=1)
            predictions.extend(predicted.cpu().numpy())
            targets.extend(label.cpu().numpy())
            
        
    predictions = np.array(predictions)
    targets = np.array(targets)
    print(f"Test Accuracy of the model on the test images: {accuracy_score(targets, predictions)*100} %")
    return predictions, targets

In [0]:
predictions, targets = test(model, dataloaders["test"])

In [0]:
ids = np.where((np.array(predictions) != np.array(targets)))[0]
num = 310
image, label = datasets["test"][ids[num]]
print(image.shape, label)
plt.title(f"Real: {classes[targets[ids[num]]]}. Predicted: {classes[predictions[ids[num]]]}")
plt.imshow(image.numpy().squeeze(), cmap="gray")

In [0]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    # print(cm)
    plt.figure(figsize=(20,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()


In [0]:
# построим confusion matrix, чтобы понять, где ошибается сеть

cm = confusion_matrix(targets, predictions)
plot_confusion_matrix(cm, classes, normalize=False)

# Задание

Добавьте в архитектуру сверточной сети еще один сверточной сети так, чтобы повысилось качество обучения

# Ура! Мы обучили двухслойную нейронную сеть!

# А вот какие сети являются одними из самых передовых на сегодняшний день

![alt text](https://neurohive.io/wp-content/uploads/2019/01/resnet-architecture-3.png)

# Очень много словев, очень долгое обучение, нужно очень много вычислительных ресурсов